In [ ]:
! pip3 install matplotlib transforms3d future typing numpy quadpy\
             numpy-stl==2.16.3 h5py sympy==1.5.1 termcolor psutil\
             symengine==0.6.1 numba Cython chaospy torch_optimizer\
             vtk chaospy termcolor omegaconf hydra-core==1.1.1 einops\
             timm tensorboard pandas orthopy ndim functorch pint

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir modulus
! cp /content/drive/MyDrive/modulus.7z /content/modulus/modulus.7z

In [ ]:
#! sudo apt install p7zip-full

In [ ]:
! 7z x /content/modulus/modulus.7z -o/content/modulus

In [ ]:
%cd /content/modulus/modulus

In [ ]:
! python setup.py install

In [ ]:
! mkdir /content/modulus_example
! mkdir /content/modulus_example/outputs
! mkdir /content/modulus_example/openfoam
! mkdir /content/modulus_example/conf
! cp /content/drive/MyDrive/Modulus_Chip2D/config_inverse.yaml /content/modulus_example/conf/config_inverse.yaml
! cp /content/drive/MyDrive/Modulus_Chip2D/2D_chip_fluid0.csv /content/modulus_example/openfoam/2D_chip_fluid0.csv
! cp /content/drive/MyDrive/Modulus_Chip2D/chip_2d_geom.png /content/modulus_example/chip_2d_geom.png

In [ ]:
%cd /content/modulus_example

The problem is defined as follows:

A 2D chip is placed inside a 2D channel. The flow enters the inlet (a parabolic profile is used with $u_{max}=1.5\text{ m/s}$) and exits through the outlet which is a $0 Pa$. All the other walls are treated as no-slip. The kinematic viscosity $(\nu)$ for the flow is $0.02 \text{ }m^2/s$ and the density $(\rho)$ is $1 \text{ }kg/m^3$. The problem is shown in the figure below.

<img src="/content/modulus_example/chip_2d_geom.png" alt="Drawing" style="width: 800px;"/>

<img src="https://drive.google.com/uc?id=1Hln7X2cg6-Psw5WrVvUDROygI1fTzNpM" alt="Drawing" style="width: 800px;"/>




In [ ]:
import numpy as np
from sympy import Symbol, Eq

import torch
import numpy as np
from sympy import Symbol, Eq

import modulus
from modulus.hydra import to_absolute_path, ModulusConfig, instantiate_arch
from modulus.utils.io import csv_to_dict
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_2d import Rectangle, Line, Channel2D
from modulus.utils.sympy.functions import parabola
from modulus.eq.pdes.navier_stokes import NavierStokes
from modulus.eq.pdes.basic import NormalDotVec
from modulus.domain.constraint import (
    PointwiseConstraint,
)
from modulus.domain.monitor import PointwiseMonitor
from modulus.key import Key
from modulus.node import Node
from modulus.domain.validator import PointwiseValidator
from modulus.key import Key
from modulus.node import Node
from modulus.utils.io.plotter import ValidatorPlotter


In [ ]:
from modulus.hydra.utils import compose, to_yaml

In [ ]:
cfg = compose(config_path="conf", config_name="config_inverse")

In [ ]:
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

In [ ]:
# make list of nodes to unroll graph on
ns = NavierStokes(nu="nu", rho=1.0, dim=2, time=False)
flow_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u"), Key("v"), Key("p")],
    cfg=cfg.arch.fully_connected,
)
inverse_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("nu")],
    cfg=cfg.arch.fully_connected,
)

nodes = (
    ns.make_nodes(
        detach_names=["u",
                      "u__x",
                      "u__x__x",
                      "u__y",
                      "u__y__y",
                      "v",
                      "v__x",
                      "v__x__x",
                      "v__y",
                      "v__y__y",
                      "p",
                      "p__x",
                      "p__y",
                      ]
    )
    + [flow_net.make_node(name="flow_network", jit=cfg.jit)]
    + [inverse_net.make_node(name="inverse_network", jit=cfg.jit)]
)

# add constraints to solver
# data
mapping = {"Points:0": "x", "Points:1": "y", "U:0": "u", "U:1": "v", "p": "p"}
openfoam_var = csv_to_dict(to_absolute_path("openfoam/2D_chip_fluid0.csv"), mapping)
openfoam_var["x"] -= 2.5  # normalize pos
openfoam_var["y"] -= 0.5
openfoam_invar_numpy = {
    key: value for key, value in openfoam_var.items() if key in ["x", "y"]
}
openfoam_outvar_numpy = {
    key: value for key, value in openfoam_var.items() if key in ["u", "v", "p"]
}
openfoam_outvar_numpy["continuity"] = np.zeros_like(openfoam_outvar_numpy["u"])
openfoam_outvar_numpy["momentum_x"] = np.zeros_like(openfoam_outvar_numpy["u"])
openfoam_outvar_numpy["momentum_y"] = np.zeros_like(openfoam_outvar_numpy["u"])

# make domain
domain = Domain()

# data and pdes
data = PointwiseConstraint.from_numpy(
    nodes=nodes,
    invar=openfoam_invar_numpy,
    outvar=openfoam_outvar_numpy,
    batch_size=cfg.batch_size.data,
)
domain.add_constraint(data, name="Data")

# add monitors
monitor = PointwiseMonitor(
    openfoam_invar_numpy,
    output_names=["nu"],
    metrics={"mean_nu": lambda var: torch.mean(var["nu"])},
    nodes=nodes,
)
domain.add_monitor(monitor)


In [ ]:
%tensorboard --logdir outputs/

In [ ]:
# make solver
slv = Solver(cfg, domain)



In [ ]:
# start solver
slv.solve()